## Part V: Training a Model with Data Programming

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ features snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`, feature matrix, and label matrix

In [4]:
from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [5]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

%time F_train = feature_manager.load(session, train, 'Train Features')

CPU times: user 2 s, sys: 64.1 ms, total: 2.06 s
Wall time: 2.1 s


In [6]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

%time L_train = label_manager.load(session, train, 'LF Labels')

CPU times: user 46 ms, sys: 3.28 ms, total: 49.2 ms
Wall time: 66.4 ms


## Train Generative Model

In [7]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=3000, rate=1e-5)

Training marginals (!= 0.5):	319
Features:			13
Begin training for rate=1e-05, mu=1e-06
	Learning epoch = 0	Gradient mag. = 0.383645
	Learning epoch = 250	Gradient mag. = 0.403600
	Learning epoch = 500	Gradient mag. = 0.403362
	Learning epoch = 750	Gradient mag. = 0.403126
	Learning epoch = 1000	Gradient mag. = 0.402889
	Learning epoch = 1250	Gradient mag. = 0.402652
	Learning epoch = 1500	Gradient mag. = 0.402416
	Learning epoch = 1750	Gradient mag. = 0.402179
	Learning epoch = 2000	Gradient mag. = 0.401943
	Learning epoch = 2250	Gradient mag. = 0.401707
	Learning epoch = 2500	Gradient mag. = 0.401471
	Learning epoch = 2750	Gradient mag. = 0.401236
Final gradient magnitude for rate=1e-05, mu=1e-06: 0.401


In [8]:
gen_model.save(session, 'Generative Params')

In [9]:
train_marginals = gen_model.marginals(L_train)

## Train Discriminative Model

In [10]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=5000, rate=1e-3)

Training marginals (!= 0.5):	232
Features:			11085
Using gradient descent...
	Learning epoch = 0	Step size = 0.001
	Loss = 160.810146	Gradient magnitude = 249.384722
	Learning epoch = 100	Step size = 0.000904792147114
	Loss = 119.877978	Gradient magnitude = 163.152286
	Learning epoch = 200	Step size = 0.000818648829479
	Loss = 114.811929	Gradient magnitude = 18.324685
	Learning epoch = 300	Step size = 0.000740707032156
	Loss = 114.729231	Gradient magnitude = 0.405607
	Learning epoch = 400	Step size = 0.000670185906007
	Loss = 114.720358	Gradient magnitude = 0.310139
	Learning epoch = 500	Step size = 0.000606378944861
	Loss = 114.715288	Gradient magnitude = 0.256367
	Learning epoch = 600	Step size = 0.000548646907485
	Loss = 114.711999	Gradient magnitude = 0.222457
	Learning epoch = 700	Step size = 0.000496411413431
	Loss = 114.709685	Gradient magnitude = 0.199401
	Learning epoch = 800	Step size = 0.00044914914861
	Loss = 114.707963	Gradient magnitude = 0.182861
	Learning epoch = 900	St

In [11]:
disc_model.w.shape

(11085,)

In [12]:
%time disc_model.save(session, "Discriminative Params")

CPU times: user 3.06 s, sys: 81.1 ms, total: 3.14 s
Wall time: 3.06 s


## Assess Performance on Development Set

In [13]:
from snorkel.models import CandidateSet
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()

### Loading the Development Feature Matrix

In [14]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()
%time F_dev = feature_manager.load(session, dev, 'Train Features')

CPU times: user 2.01 s, sys: 51.8 ms, total: 2.06 s
Wall time: 2.11 s


In [15]:
L_dev = label_manager.load(session, dev, "Hardware Development Labels -- Gold")

In [16]:
gold_dev_set = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Gold').one()

In [17]:
F_dev.shape

(313, 11085)

In [18]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

Calibration plot:
Recall-corrected Noise-aware Model
Pos. class accuracy: 0.7
Neg. class accuracy: 0.932862190813
Corpus Precision 0.525
Corpus Recall    0.7
Corpus F1        0.6
----------------------------------------
TP: 21 | FP: 19 | TN: 264 | FN: 9

Recall-corrected Noise-aware Model
Pos. class accuracy: 0.7
Neg. class accuracy: 0.932862190813
Corpus Precision 0.525
Corpus Recall    0.7
Corpus F1        0.6
----------------------------------------
TP: 21 | FP: 19 | TN: 264 | FN: 9



/Users/bradenhancock/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [19]:
def error_analysis(c):
    print c[0]
    print c[0].parent
    print (c[0].parent.row_num, c[0].parent.col_num)
    print c[1]
    print c[1].parent
    print (c[1].parent.row_num, c[1].parent.col_num)

In [20]:
c = list(tp)[10]
error_analysis(c)
print c.temp.parent.html_anc_tags

Span("2N3905", parent=1735, chars=[26,31], words=[3,3])
Phrase(Document CSEMS03485-1, 0, u'The CENTRAL SEMICONDUCTOR 2N3905 and')
(None, None)
Span("-65", parent=101754, chars=[0,2], words=[0,0])
Phrase(Document CSEMS03485-1, 0, u'-65 to +150')
(4, 2)
['html', 'body', 'table', 'tr', 'td', 'p']


In [21]:
from snorkel.models import Corpus
from hardware_utils import entity_level_f1
import os

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
entity_level_f1(tp, fp, tn, fn, filename, corpus, 'stg_temp_min')

Scoring on Entity-Level Gold Data
Corpus Precision 0.525
Corpus Recall    0.196
Corpus F1        0.286
----------------------------------------
TP: 21 | FP: 19 | FN: 86



In [22]:
import os
os.system('cp snorkel.db snorkel.db\ final');

The End.